In [1]:
# This notebook uses the python2 kernel.

import matlab.engine

import os
os.chdir('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/Code/Library')
# Need to have the TFOCS package in Library to call the slope solver.
# Link to TFOCS: https://github.com/cvxr/TFOCS.git
# Version 1.3, October 10, 2013

from math import log, exp
import numpy as np

from scipy import stats

from collections import namedtuple

from copy import deepcopy

import matplotlib.pyplot as plt

from ExperimentFunc import exp_func, beta_gen_slope
from Step2 import find_v_slope
from Step3 import solve_omega, gw_l1, proj_l1_tan_cone, proj_l1_neg_tan_cone

In [2]:
eng = matlab.engine.start_matlab()

matpath = eng.genpath('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/Code/Library/TFOCS')
eng.addpath(matpath)

'/Users/yufei/Documents/MATLAB:/Applications/MATLAB_R2019a.app/toolbox/matlab/capabilities:/Applications/MATLAB_R2019a.app/toolbox/matlab/datafun:/Applications/MATLAB_R2019a.app/toolbox/matlab/datatypes:/Applications/MATLAB_R2019a.app/toolbox/matlab/elfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/elmat:/Applications/MATLAB_R2019a.app/toolbox/matlab/funfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/general:/Applications/MATLAB_R2019a.app/toolbox/matlab/iofun:/Applications/MATLAB_R2019a.app/toolbox/matlab/lang:/Applications/MATLAB_R2019a.app/toolbox/matlab/matfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/mvm:/Applications/MATLAB_R2019a.app/toolbox/matlab/ops:/Applications/MATLAB_R2019a.app/toolbox/matlab/polyfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/randfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/sparfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/specfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/strfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/

In [3]:
def solve_beta_slope(X, Y, param_set):
    """
    Solve the SLOPE by calling a matlab function in TFOCS package.
    
    @param X np.array(n, p): the data matrix.
    @param Y np.array(n, ): the response vector.
    @param param_set list[np.array(p, )]: contains an array of tuning param according to the 'slope meets lasso' paper
    
    @return np.array(p, ): the coefficient got by SLOPE.
    """
    Y_t = eng.transpose(matlab.double(Y.tolist()))
    lbd_t = eng.transpose(matlab.double(param_set[0].tolist()))
    beta_hat = eng.solver_SLOPE(matlab.double(X.tolist()), Y_t, lbd_t)
    return np.asarray(beta_hat).squeeze()


def obj_sqrtslope(X, Y, lbd, beta, sigma):
    """
    Evaluate the objective function in the optimization of square-root SLOPE.
    See p. 751 of 'Improved bounds for square-root lasso and square-root slope'.
    
    @param X np.array: the data matrix.
    @param Y np.array: the response vector.
    @param lbd np.array: the constraint parameter.
    @param beta np.array: the coefficient.
    @param sigma float: the standard error of noise.
    
    @return float: the value of square-root SLOPE loss function.
    """
    n = X.shape[0]
    return sigma + np.sum((Y - np.matmul(X,beta))**2)/(n * sigma) + np.sum(lbd * np.sort(abs(beta))[::-1])


def solve_beta_sqrtslope(X, Y, param_set, stop_criteria=0.01):
    """
    Solve the square-root SLOPE.
    
    @param X np.array(n, p): the data matrix.
    @param Y np.array(n, ): the response vector.
    @param param_set list[np.array(p, )]: contains an array of tuning param according to Derumigny's paper
    
    @return np.array: the coefficient got by square-root SLOPE.
    """
    n, p = X.shape[0], X.shape[1]

    sigma_prev, sigma_new = np.var(Y)**0.5, np.var(Y)**0.5
    
    beta_prev, beta_new = np.zeros(p), solve_beta_slope(X, Y, [sigma_new*param_set[0]])

    i = 1.0
    while abs(obj_sqrtslope(X, Y, param_set[0], beta_prev, sigma_prev) - obj_sqrtslope(X, Y, param_set[0], beta_new, sigma_new)) > stop_criteria:
        sigma_prev = sigma_new
        beta_prev = beta_new
        sigma_new = (sum((Y - np.matmul(X,beta_prev))**2)/n)**0.5
        beta_new = solve_beta_slope(X, Y, [sigma_new*param_set[0]])
        i += 1
        print( i, abs(obj_sqrtslope(X, Y, param_set[0], beta_prev, sigma_prev) - obj_sqrtslope(X, Y, param_set[0], beta_new, sigma_new)) )

    return beta_new

In [4]:
Params = namedtuple('Params', ['step1', 'step2', 'step3'])

### <span style="color:purple">1) Cov(X) = I</span>

In [31]:
N = 100
n = 1000
p = 1000
Sigma_sqrt = np.eye(p)
noise_sd = 2
debias_idx = p - 1

cardi = 0.005   # number of non-zero coordinates
su = int(p * cardi)        # upper bound of number of non-zero coordinates

# construct lambda vector
A = (19) * (16 + 4 * 2**0.5) * 2   # the constant used to form lambda in constraint. Tuned. Don't touch! The estimation of sigma is super sensitive about this.
lbd_vec = A * (np.log(2*p/np.arange(1,p+1))/n)**0.5

C = (su * log(exp(1)*p/su) / n**0.5)**(-1/3)   # the constant in the bound of L2 error

param_set = Params([lbd_vec], 
                   [C, su], 
                   [gw_l1, proj_l1_tan_cone, proj_l1_neg_tan_cone])

In [10]:
z, z_biased = exp_func(N,
             n,
             p, 
             Sigma_sqrt, 
             noise_sd, 
             debias_idx,
             param_set, 
             beta_gen_slope, 
             solve_beta_sqrtslope, 
             find_v_slope, 
             solve_omega)

('iter:', 0)
(2.0, 210.45591422955988)
(3.0, 58.74018555275018)
(4.0, 6.600913402953665)
(5.0, 2.014900217414379)
(6.0, 0.38077690068155334)
(7.0, 0.07173496651353162)
(8.0, 0.013498654605314186)
(9.0, 0.0025774971421697046)
('The L2 error: ', 0.5776240910209198)
('iter:', 1)
(2.0, 197.05976496636163)
(3.0, 39.86716705252502)
(4.0, 4.911012725617638)
(5.0, 0.874807809079698)
(6.0, 1.4869934669281974)
(7.0, 0.07939277034142833)
(8.0, 1.3459295361662953)
(9.0, 1.3903948295017017)
(10.0, 1.3931998970339237)
(11.0, 0.060621084782951584)
(12.0, 1.3555460414034997)
(13.0, 1.3004743910888692)
(14.0, 0.055865600594188436)
(15.0, 0.008011116265720375)
('The L2 error: ', 0.7576312215466414)
('iter:', 2)
(2.0, 198.07659290664674)
(3.0, 44.57533238266842)
(4.0, 10.07627522684811)
(5.0, 3.4886000439122427)
(6.0, 0.8958144116271569)
(7.0, 0.023483639470669004)
(8.0, 0.00375854021365285)
('The L2 error: ', 0.7734852854774542)
('iter:', 3)
(2.0, 198.93875437513657)
(3.0, 49.14627572622942)
(4.0, 11.84

('iter:', 31)
(2.0, 198.62917084863784)
(3.0, 41.452676243166025)
(4.0, 7.4127956704775215)
(5.0, 6.732410421671375)
(6.0, 5.172056541724146)
(7.0, 0.19123107912867)
(8.0, 0.02935320975700506)
(9.0, 0.00465120520561868)
('The L2 error: ', 0.6807530439162924)
('iter:', 32)
(2.0, 204.11387357055594)
(3.0, 54.49505242771647)
(4.0, 13.296645988023556)
(5.0, 2.861435002864482)
(6.0, 0.6171741117356078)
(7.0, 0.1323574256957727)
(8.0, 0.028346976404577617)
(9.0, 0.006237638710445026)
('The L2 error: ', 0.7855344200375632)
('iter:', 33)
(2.0, 197.5111945339164)
(3.0, 42.057681580153144)
(4.0, 2.4118919373179324)
(5.0, 0.8805535685258974)
(6.0, 0.15664061710003807)
(7.0, 0.027794882201874316)
(8.0, 0.004930699154328977)
('The L2 error: ', 0.762699649953203)
('iter:', 34)
(2.0, 209.07181600150534)
(3.0, 42.76266494643971)
(4.0, 11.841379417325811)
(5.0, 2.97215108655962)
(6.0, 0.869981011712639)
(7.0, 0.2540488956235549)
(8.0, 0.07410010304931802)
(9.0, 0.021585744079061442)
(10.0, 0.0063581812

#### Compare the mean of the (debiased_beta - beta) and (non-debiased_beta - beta)

In [11]:
mean_non_debiased = np.mean(z_biased)
print("The mean of (non_debiased_beta - beta) is: ", mean_non_debiased)

('The mean of (non_debiased_beta - beta) is: ', 3.3129888878614624)


In [12]:
mean_debiased = np.mean(np.array(z))
print("The mean of (debiased_beta - beta) is: ", mean_debiased)

('The mean of (debiased_beta - beta) is: ', -0.558760813919676)


#### Save the simulation results

In [18]:
np.save('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/ExpResults/SqrtSLOPE/identity_z_biased.npy', z_biased)
np.save('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/ExpResults/SqrtSLOPE/identity_z.npy', z)

###  <span style="color:purple">2) Cov(X) with bounded eigenvalues</span>

In [20]:
# other parameters are the same as cov=I case

# Generate a cov matrix with bounded eigenvalues
# generate eigenvalues
cov_eigv = np.random.uniform(low = 0.5, high = 3.0, size = (p,))
D_sqrt = np.diag(cov_eigv**0.5)
# generate an orthonormal matrix
a = np.random.normal(size = (p,p))
u, s, vh = np.linalg.svd(np.matmul(a.T,a), full_matrices=True)
# generate the square root of cov matrix 
Sigma_sqrt = np.matmul(D_sqrt,u.T)

In [21]:
z, z_biased = exp_func(N,
             n,
             p, 
             Sigma_sqrt, 
             noise_sd, 
             debias_idx,
             param_set, 
             beta_gen_slope, 
             solve_beta_sqrtslope, 
             find_v_slope, 
             solve_omega)

('iter:', 0)
(2.0, 175.76428839126004)
(3.0, 86.25239315411636)
(4.0, 45.67651193815095)
(5.0, 24.760078401962573)
(6.0, 13.786749209768914)
(7.0, 7.587782475001404)
(8.0, 4.229429576089387)
(9.0, 2.3922405695655016)
(10.0, 1.3490055958457106)
(11.0, 0.7637014565964364)
(12.0, 0.43079375972388334)
(13.0, 0.24285728978088628)
(14.0, 0.1372422765527972)
(15.0, 0.07725569012018241)
(16.0, 0.04349722022698188)
(17.0, 0.02449324262238406)
(18.0, 0.013794492658462332)
(19.0, 0.00776396492551612)
('The L2 error: ', 0.639894784339706)
('iter:', 1)
(2.0, 182.51057595767134)
(3.0, 72.84247768895625)
(4.0, 30.828624818942103)
(5.0, 13.21351818603921)
(6.0, 5.588167783311519)
(7.0, 2.423447994780645)
(8.0, 1.0880359053064694)
(9.0, 0.4818912756265945)
(10.0, 0.21344601355190207)
(11.0, 0.09446899252225194)
(12.0, 0.04187915013449128)
(13.0, 0.018638329898749362)
(14.0, 0.00831010950378186)
('The L2 error: ', 0.49006679156669297)
('iter:', 2)
(2.0, 177.33847014390824)
(3.0, 87.95565602934107)
(4.0,

(16.0, 0.015367454961278781)
(17.0, 0.007969731065713859)
('The L2 error: ', 0.4703550615206118)
('iter:', 18)
(2.0, 180.42489795659287)
(3.0, 86.45330490990227)
(4.0, 39.22350512909907)
(5.0, 17.981904226606275)
(6.0, 8.560558530393337)
(7.0, 4.03066921199752)
(8.0, 1.9311198739092106)
(9.0, 0.9316454685435929)
(10.0, 0.45274580255363617)
(11.0, 0.2217853563722656)
(12.0, 0.10861583345581494)
(13.0, 0.053205398716499985)
(14.0, 0.026097148860344532)
(15.0, 0.012787315738705729)
(16.0, 0.006269647180943139)
('The L2 error: ', 0.4575451601884344)
('iter:', 19)
(2.0, 184.23774159829486)
(3.0, 76.85650592541185)
(4.0, 36.431787421997)
(5.0, 18.13547854420358)
(6.0, 9.423610561446822)
(7.0, 5.0214053881120435)
(8.0, 2.6928023345979)
(9.0, 1.4390738622496428)
(10.0, 0.7681696705762988)
(11.0, 0.4101633897200827)
(12.0, 0.21807213359215893)
(13.0, 0.11571705441156155)
(14.0, 0.06134661840019362)
(15.0, 0.032519609549353845)
(16.0, 0.01722117643839738)
(17.0, 0.009143420283407977)
('The L2 er

(3.0, 69.24027234932737)
(4.0, 31.382639503547125)
(5.0, 15.128398983811394)
(6.0, 7.305496297119362)
(7.0, 3.5836663476738977)
(8.0, 1.7890166856404903)
(9.0, 0.9040500018281818)
(10.0, 0.4574319229454886)
(11.0, 0.23121604023822329)
(12.0, 0.11711023372822638)
(13.0, 0.0594052555172766)
(14.0, 0.029995931252869923)
(15.0, 0.015106705554444488)
(16.0, 0.007569169719090496)
('The L2 error: ', 0.536909660574153)
('iter:', 36)
(2.0, 181.51106523371118)
(3.0, 94.60600181251766)
(4.0, 47.27790957054549)
(5.0, 24.966589339613392)
(6.0, 13.183286746180556)
(7.0, 6.946225757425964)
(8.0, 3.621356511670683)
(9.0, 1.8767865618642645)
(10.0, 0.9761233633728352)
(11.0, 0.5129705330143679)
(12.0, 0.2709188229046049)
(13.0, 0.14435734809694623)
(14.0, 0.07727950034814057)
(15.0, 0.04135064204115224)
(16.0, 0.022117940064845243)
(17.0, 0.011831848903511855)
(18.0, 0.006335058257718629)
('The L2 error: ', 0.5696547901146384)
('iter:', 37)
(2.0, 191.5525093007983)
(3.0, 90.21912969015511)
(4.0, 40.576

(14.0, 0.03817869199406232)
(15.0, 0.019577633485823753)
(16.0, 0.010013226843739176)
(17.0, 0.005123343583818496)
('The L2 error: ', 0.4739490751293583)
('iter:', 53)
(2.0, 168.61960973109228)
(3.0, 102.59901702735363)
(4.0, 49.20891354311675)
(5.0, 25.68952825302449)
(6.0, 13.842221357863309)
(7.0, 7.7338509787559815)
(8.0, 4.244049031330405)
(9.0, 2.4482700296077837)
(10.0, 1.4342154319547262)
(11.0, 0.8413714889520634)
(12.0, 0.49021753524039013)
(13.0, 0.2843549978572355)
(14.0, 0.1649977729280181)
(15.0, 0.09570600776396532)
(16.0, 0.05494047646789113)
(17.0, 0.031524273197646835)
(18.0, 0.018121670566870307)
(19.0, 0.010384463094851526)
(20.0, 0.0059290190106366936)
('The L2 error: ', 0.5942392752816176)
('iter:', 54)
(2.0, 172.517909920109)
(3.0, 86.9848925955423)
(4.0, 43.52872877198115)
(5.0, 21.614608769454435)
(6.0, 10.784606018464956)
(7.0, 5.486985895000998)
(8.0, 2.767780099942456)
(9.0, 1.3913529425974502)
(10.0, 0.7036415604545709)
(11.0, 0.35824257771628254)
(12.0, 0.

(13.0, 0.09487538967414366)
(14.0, 0.05095425341983173)
(15.0, 0.027351112164296865)
(16.0, 0.014686952107695106)
(17.0, 0.007886380687750716)
('The L2 error: ', 0.5249061462220064)
('iter:', 70)
(2.0, 180.87518813454267)
(3.0, 88.1925374455002)
(4.0, 43.408608502333664)
(5.0, 22.003364545142404)
(6.0, 11.59183839283969)
(7.0, 6.151447215922417)
(8.0, 3.206418272321571)
(9.0, 1.678809089176866)
(10.0, 0.8866216826627351)
(11.0, 0.46930023277195687)
(12.0, 0.2516707745353415)
(13.0, 0.13774991819195748)
(14.0, 0.07537636736969944)
(15.0, 0.04124324890085518)
(16.0, 0.022557323316732436)
(17.0, 0.01234278510241893)
(18.0, 0.0067545306578722375)
('The L2 error: ', 0.47553903559961036)
('iter:', 71)
(2.0, 173.96681648501612)
(3.0, 88.0648466651462)
(4.0, 41.79208712950367)
(5.0, 19.96873784525269)
(6.0, 9.518029746179536)
(7.0, 4.773750150419346)
(8.0, 2.395779057943173)
(9.0, 1.2165444156398735)
(10.0, 0.6152378040740132)
(11.0, 0.3102720589185992)
(12.0, 0.15638966344999972)
(13.0, 0.078

(8.0, 4.057649051343105)
(9.0, 2.2590563983640095)
(10.0, 1.269209272409853)
(11.0, 0.7166278214706381)
(12.0, 0.41061414943806085)
(13.0, 0.23698977532467325)
(14.0, 0.13693472096315418)
(15.0, 0.07905366443992534)
(16.0, 0.04563227208518583)
(17.0, 0.026339247159512524)
(18.0, 0.015203668216599908)
(19.0, 0.008764247292674554)
('The L2 error: ', 0.5498677496165572)
('iter:', 88)
(2.0, 176.44472344394137)
(3.0, 78.52297668699134)
(4.0, 39.89092004059694)
(5.0, 20.14695752131047)
(6.0, 10.090143989853004)
(7.0, 5.197280963150661)
(8.0, 2.7006649383793047)
(9.0, 1.3741374456737958)
(10.0, 0.6986510175004241)
(11.0, 0.3537281838889612)
(12.0, 0.17868821396632484)
(13.0, 0.09017416334904738)
(14.0, 0.045511782105222665)
(15.0, 0.022969993467313543)
(16.0, 0.011577819158901548)
(17.0, 0.005845591852903453)
('The L2 error: ', 0.5282305195065063)
('iter:', 89)
(2.0, 175.06172790230357)
(3.0, 74.09193833246798)
(4.0, 34.36013702933246)
(5.0, 16.866822053223586)
(6.0, 8.2717922502884)
(7.0, 4.

#### Compare the mean of the (debiased_beta - beta) and (non-debiased_beta - beta)

In [22]:
mean_non_debiased = np.mean(z_biased)
print("The mean of (non_debiased_beta - beta) is: ", mean_non_debiased)

('The mean of (non_debiased_beta - beta) is: ', 2.8147046904315163)


In [23]:
mean_debiased = np.mean(np.array(z))
print("The mean of (debiased_beta - beta) is: ", mean_debiased)

('The mean of (debiased_beta - beta) is: ', -0.5074283188098104)


#### Save the simulation results

In [24]:
np.save('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/ExpResults/SqrtSLOPE/bddeig_z_biased.npy', z_biased)
np.save('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/ExpResults/SqrtSLOPE/bddeig_z.npy', z)

### <span style = 'color:purple'>3) Cov(X) is the Cov of AR(1) Process</span>

In [32]:
# other parameters are the same as cov=I case

# Generate the squar root of cov matrix
rho = 0.4
rho_vec = []
for i in range(p):
    rho_vec.append(rho**i)
rho_vec = np.array(rho_vec)
# The cholesky decomposition of cov == the squar root of cov
Sigma_sqrt = [rho_vec]
for i in range(1, p):
    rho_vec_shifted = np.concatenate((np.zeros(i), rho_vec[:-i]))
#     print(rho_vec_shifted)
    Sigma_sqrt.append(rho_vec_shifted * (1-rho**2)**0.5)
Sigma_sqrt = np.array(Sigma_sqrt)

In [33]:
z, z_biased = exp_func(N,
             n,
             p, 
             Sigma_sqrt, 
             noise_sd, 
             debias_idx,
             param_set, 
             beta_gen_slope, 
             solve_beta_sqrtslope, 
             find_v_slope, 
             solve_omega)

('iter:', 0)
(2.0, 136.5513333739483)
(3.0, 22.655662219937312)
(4.0, 3.182294033144103)
(5.0, 0.37736458753158786)
(6.0, 0.04408582383621251)
(7.0, 0.005130701925054382)
('The L2 error: ', 0.3125981717699184)
('iter:', 1)
(2.0, 127.84737967792807)
(3.0, 16.945532071365506)
(4.0, 1.5868288338965613)
(5.0, 0.12586541825942277)
(6.0, 0.00988478217834654)
('The L2 error: ', 0.2971576404591255)
('iter:', 2)
(2.0, 121.99828414404249)
(3.0, 18.762907983833998)
(4.0, 2.7946265088480686)
(5.0, 0.37751344915648133)
(6.0, 0.04982235472550656)
(7.0, 0.0065527190035936655)
('The L2 error: ', 0.6194554668322084)
('iter:', 3)
(2.0, 121.23906096525423)
(3.0, 16.010389943631708)
(4.0, 1.2407761216765039)
(5.0, 0.08252136166561286)
(6.0, 0.005435672926864754)
('The L2 error: ', 0.5698709044893356)
('iter:', 4)
(2.0, 130.17830862025994)
(3.0, 26.708001977076492)
(4.0, 4.67394328998148)
(5.0, 0.7199800160740324)
(6.0, 0.10925163448541753)
(7.0, 0.01654209684693342)
(8.0, 0.0024961182257356995)
('The L2 e

(5.0, 0.031232596854465555)
(6.0, 0.0013893613511299918)
('The L2 error: ', 0.513280370154701)
('iter:', 41)
(2.0, 129.13778822643587)
(3.0, 18.07385697599875)
(4.0, 2.0476109630684505)
(5.0, 0.19967980006970265)
(6.0, 0.019273254561198883)
(7.0, 0.001858161886389098)
('The L2 error: ', 0.4333190829593896)
('iter:', 42)
(2.0, 127.08440431476686)
(3.0, 19.083692874157805)
(4.0, 1.822447247639957)
(5.0, 0.17453287928685768)
(6.0, 0.016524195645047257)
(7.0, 0.0015637502764320743)
('The L2 error: ', 0.31337955880613616)
('iter:', 43)
(2.0, 125.75883543685825)
(3.0, 22.407105398419276)
(4.0, 2.488722473153757)
(5.0, 0.23063195459371855)
(6.0, 0.021071383373737262)
(7.0, 0.0019282841594758793)
('The L2 error: ', 0.5332253063345037)
('iter:', 44)
(2.0, 127.99128927697461)
(3.0, 13.455933223628449)
(4.0, 0.9126985038225257)
(5.0, 0.05174969638392213)
(6.0, 0.0029092662701941663)
('The L2 error: ', 0.4067566504234451)
('iter:', 45)
(2.0, 123.93628792400591)
(3.0, 16.276777784765727)
(4.0, 1.57

(4.0, 1.857403222226253)
(5.0, 0.17805295969219515)
(6.0, 0.016908231728621104)
(7.0, 0.0016093256565454794)
('The L2 error: ', 0.2875427022618331)
('iter:', 81)
(2.0, 128.00830845904477)
(3.0, 17.004427095057395)
(4.0, 1.779992343567983)
(5.0, 0.18520440687711925)
(6.0, 0.019076927185778914)
(7.0, 0.001965015818655047)
('The L2 error: ', 0.34159301299046063)
('iter:', 82)
(2.0, 132.15063238928542)
(3.0, 22.359793909480004)
(4.0, 4.305736004917776)
(5.0, 0.7340258571966842)
(6.0, 0.12371404985367462)
(7.0, 0.02081101619467063)
(8.0, 0.003497532861047148)
('The L2 error: ', 0.38396144904119595)
('iter:', 83)
(2.0, 128.74842913337716)
(3.0, 13.219203662222867)
(4.0, 0.6677403405641371)
(5.0, 0.027524241710352726)
(6.0, 0.0011310712186514138)
('The L2 error: ', 0.3272648744861239)
('iter:', 84)
(2.0, 115.41398235210988)
(3.0, 11.646418107857698)
(4.0, 0.7659653774729804)
(5.0, 0.04501475644599395)
(6.0, 0.0026225144918043952)
('The L2 error: ', 0.4221176326713077)
('iter:', 85)
(2.0, 130.

#### Compare the mean of the (debiased_beta - beta) and (non-debiased_beta - beta)

In [34]:
mean_non_debiased = np.mean(z_biased)
print("The mean of (non_debiased_beta - beta) is: ", mean_non_debiased)

('The mean of (non_debiased_beta - beta) is: ', 1.1082941297734172)


In [35]:
mean_debiased = np.mean(np.array(z))
print("The mean of (debiased_beta - beta) is: ", mean_debiased)

('The mean of (debiased_beta - beta) is: ', -0.2784931830622827)


#### Save the simulation results

In [29]:
np.save('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/ExpResults/SqrtSLOPE/ar1_z_biased2.npy', z_biased)
np.save('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/ExpResults/SqrtSLOPE/ar1_z2.npy', z)